In [ ]:
#this is the code for identifying  keystone species

#question 1 for implentation: How is the proposer going to work?

explore using mult-arm bandits or hyperparameter optimization

bayesian information criteria is the reward. the accuracy aspect of BIC will be defined as how close to the data it is. 

the data used will be MEG or EEG. 

the policies will be trained normally. number of groupings of agents will be a hyperparameter. these instances will be trained in parallel fashion

A2c will be used. 

A2C will be used because it is decentralized and only algorithms with decentralized execution support groupings.

LAter MAPDDG could also be used since it is centralized learning with decentralized execution 

the number of groupings will increase until and be compared to a smaller number of groupings until reward stops improving. 

libraries used:

RLLIB 
rl-warpdrive

Primary: 

rl-warpdrive

We will have to switch the architecture to PPO. 

In [1]:
from ray.rllib.connectors.env_to_module import FlattenObservations
from ray.rllib.core.rl_module.multi_rl_module import MultiRLModuleSpec
from ray.rllib.core.rl_module.rl_module import RLModuleSpec
from ray.rllib.examples.envs.classes.two_step_game import TwoStepGameWithGroupedAgents
from ray.rllib.utils.test_utils import (
    add_rllib_example_script_args,
    run_rllib_example_script_experiment,
)
from ray.tune.registry import register_env, get_trainable_cls

from example_envs.tag_continuous.tag_continuous import TagContinuous
from warp_drive.env_wrapper import EnvWrapper
from warp_drive.training.trainer import Trainer

pygame 2.5.1 (SDL 2.28.2, Python 3.10.11)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
run_config = dict(
    name="tag_continuous",
    # Environment settings
    env=dict(
        num_taggers=5,
        num_runners=20,
        episode_length=100,
        seed=1234,
        use_full_observation=False,
        num_other_agents_observed=10,
        tagging_distance=0.02,
    ),
    # Trainer settings
    trainer=dict(
        num_envs=100,  # number of environment replicas (number of GPU blocks used)
        train_batch_size=10000,  # total batch size used for training per iteration (across all the environments)
        num_episodes=5000,  # total number of episodes to run the training for (can be arbitrarily high!)
    ),
    # Policy network settings
    policy=dict(
        runner=dict(
            to_train=True,  # flag indicating whether the model needs to be trained
            algorithm="A2C",  # algorithm used to train the policy
            gamma=0.98,  # discount rate
            lr=0.005,  # learning rate
            model=dict(
                type="fully_connected", fc_dims=[256, 256], model_ckpt_filepath=""
            ),  # policy model settings
        ),
        tagger=dict(
            to_train=True,
            algorithm="A2C",
            gamma=0.98,
            lr=0.002,
            model=dict(
                type="fully_connected", fc_dims=[256, 256], model_ckpt_filepath=""
            ),
        ),
    ),
    # Checkpoint saving setting
    saving=dict(
        metrics_log_freq=10,  # how often (in iterations) to print the metrics
        model_params_save_freq=5000,  # how often (in iterations) to save the model parameters
        basedir="/tmp",  # base folder used for saving
        name="continuous_tag",  # experiment name
        tag="example",  # experiment tag
    ),
)

In [4]:
print(run_config)

{'name': 'tag_continuous', 'env': {'num_taggers': 5, 'num_runners': 20, 'episode_length': 100, 'seed': 1234, 'use_full_observation': False, 'num_other_agents_observed': 10, 'tagging_distance': 0.02}, 'trainer': {'num_envs': 100, 'train_batch_size': 10000, 'num_episodes': 5000}, 'policy': {'runner': {'to_train': True, 'algorithm': 'A2C', 'gamma': 0.98, 'lr': 0.005, 'model': {'type': 'fully_connected', 'fc_dims': [256, 256], 'model_ckpt_filepath': ''}}, 'tagger': {'to_train': True, 'algorithm': 'A2C', 'gamma': 0.98, 'lr': 0.002, 'model': {'type': 'fully_connected', 'fc_dims': [256, 256], 'model_ckpt_filepath': ''}}}, 'saving': {'metrics_log_freq': 10, 'model_params_save_freq': 5000, 'basedir': '/tmp', 'name': 'continuous_tag', 'tag': 'example'}}


In [ ]:
numPolicies= 3
for i in numPolicies:
    run_config['policy'][str(i)] = dict(
            num_envs=100,  # number of environment replicas (number of GPU blocks used)
            train_batch_size=10000,  # total batch size used for training per iteration (across all the environments)
            num_episodes=5000,  # total number of episodes to run the training for (can be arbitrarily high!)
        ),

In [ ]:
"""

for i in num_policies:
    i = dict(
            to_train=True,  # flag indicating whether the model needs to be trained
            algorithm="A2C",  # algorithm used to train the policy
            gamma=0.98,  # discount rate
            lr=0.005,  # learning rate
            model=dict(
                type="fully_connected", fc_dims=[256, 256], model_ckpt_filepath=""
            ),  # policy model settings
        ),
    policy_dict.append(i)
run_config = dict(
    name="tag_continuous",
    # Environment settings
    env=dict(
        num_taggers=5,
        num_runners=20,
        episode_length=100,
        seed=1234,
        use_full_observation=False,
        num_other_agents_observed=10,
        tagging_distance=0.02,
    ),
    # Trainer settings
    trainer=dict(
        num_envs=100,  # number of environment replicas (number of GPU blocks used)
        train_batch_size=10000,  # total batch size used for training per iteration (across all the environments)
        num_episodes=5000,  # total number of episodes to run the training for (can be arbitrarily high!)
    ),
    # Policy network settings
    policy=dict(
        



"""

modify this enviroment



class CichyEnv(MultiAgentEnv):
    def __init__(self, images, expert_rdms):
        self.images = images
        self.expert_rdms = expert_rdms
        self.num_agents = 2
        self.agent_ids = ["IT", "EVC"]
        
        env_config = {"env_name":'cichy'}


        self.observation_space = spaces.Dict({
            "image": spaces.Box(low=0, high=255, shape=(175, 175, 3), dtype=np.uint8),#this shape should actually probably be 175, 175,3
            "other_action": spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
        })

        # Continuous: 1. expending activity units 2. sending signal
        self.action_space = spaces.Box(low=-1, high=1, shape=(2,), dtype=np.float32)

        self.action_space = spaces.Dict({ 
            "IAU": spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32),
            "other_action": spaces.Box(low=-1, high=1, shape=(1,), dtype=np.float32)
            }
        )
    def reset(self):
        self.current_step = 0
        self.state = {agent: {"image": self.images[self.current_step], "other_action": np.array([0.0])} for agent in self.agent_ids}
        return self.state

    def step(self, action_dict):
        obs, rewards, dones, infos = {}, {}, {}, {}
        self.current_step += 1

        for agent_id in self.agent_ids:
            other_agent_id = "EVC" if agent_id == "IT" else "IT"
            obs[agent_id] = {"image": self.images[self.current_step], "other_action": action_dict[other_agent_id][1]}
            rewards[agent_id] = self._calculate_reward(agent_id, action_dict[agent_id])
            dones[agent_id] = self.current_step >= len(self.images) - 1
            infos[agent_id] = {}

        dones["__all__"] = all(dones.values())
        return obs, rewards, dones, infos
    def _calculate_rewards(self):
        rewards = {}
        """
        this function iterates over agents and withen each agent iterates over images to create a cosine similarity. it then calculates the reward
        for each agent 
        
        this is configured to do one pass where it generates simulated RDM for one subject
        
        """
        for agent_id in self.agent_ids:
            actions = np.array(self.actions[agent_id])
            num_images = len(self.images)
            simulated_rdm = np.zeros((num_images, num_images))#

            for i in range(num_images):
                for j in range(num_images):
                    if i != j:
                        sim = cosine_similarity(actions[i].reshape(1, -1), actions[j].reshape(1, -1))[0][0]
                        simulated_rdm[i, j] = 1 - sim

            expert_rdm = self.expert_rdms[agent_id]
            rewards[agent_id] = -np.sum((simulated_rdm - expert_rdm) ** 2)
        return rewards

example of rl-warpdrive enviroment

In [ ]:
import copy
import heapq

import numpy as np
from gym import spaces

from warp_drive.utils.constants import Constants
from warp_drive.utils.data_feed import DataFeed
from warp_drive.utils.gpu_environment_context import CUDAEnvironmentContext

_OBSERVATIONS = Constants.OBSERVATIONS
_ACTIONS = Constants.ACTIONS
_REWARDS = Constants.REWARDS
_LOC_X = "loc_x"
_LOC_Y = "loc_y"
_SP = "speed"
_DIR = "direction"
_ACC = "acceleration"
_SIG = "still_in_the_game"


class TagContinuous(CUDAEnvironmentContext):
    """
    The game of tag on a continuous circular 2D space.
    There are some taggers trying to tag several runners.
    The taggers want to get as close as possible to the runner, while the runner
    wants to get as far away from them as possible.
    Once a runner is tagged, he exits the game if runner_exits_game_after_tagged is True
    otherwise he continues to run around (and the tagger can catch him again)
    """

    def __init__(
        self,
        num_taggers=1,
        num_runners=10,
        grid_length=10.0,
        episode_length=100,
        starting_location_x=None,
        starting_location_y=None,
        starting_directions=None,
        seed=None,
        max_speed=1.0,
        skill_level_runner=1.0,
        skill_level_tagger=1.0,
        max_acceleration=1.0,
        min_acceleration=-1.0,
        max_turn=np.pi / 2,
        min_turn=-np.pi / 2,
        num_acceleration_levels=10,
        num_turn_levels=10,
        edge_hit_penalty=-0.0,
        use_full_observation=True,
        num_other_agents_observed=2,
        tagging_distance=0.01,
        tag_reward_for_tagger=1.0,
        step_penalty_for_tagger=-0.0,
        tag_penalty_for_runner=-1.0,
        step_reward_for_runner=0.0,
        end_of_game_reward_for_runner=1.0,
        runner_exits_game_after_tagged=True,
        env_backend="cpu",
    ):
        """
        Args:
            num_taggers (int, optional): [number of taggers in the environment].
                Defaults to 1.
            num_runners (int, optional): [number of taggers in the environment].
                Defaults to 10.
            grid_length (float, optional): [length of the square grid]. Defaults to 10.0
            episode_length (int, optional): [episode length]. Defaults to 100.
            starting_location_x ([ndarray], optional): [starting x locations of the
                agents]. Defaults to None.
            starting_location_y ([ndarray], optional): [starting y locations of the
            agents]. Defaults to None.
            starting_directions ([ndarray], optional): starting orientations
                in [0, 2*pi]. Defaults to None.
            seed ([type], optional): [seeding parameter]. Defaults to None.
            max_speed (float, optional): [max speed of the agents]. Defaults to 1.0
            skill_level_runner (float, optional): [runner skill level;
                this essentially is a multiplier to the max_speed].
                Defaults to 1.0
            skill_level_tagger (float, optional): [tagger skill level]. Defaults to 1.0
            max_acceleration (float, optional): [the max acceleration]. Defaults to 1.0.
            min_acceleration (float, optional): [description]. Defaults to -1.0
            max_turn ([type], optional): [description]. Defaults to np.pi/2.
            min_turn ([type], optional): [description]. Defaults to -np.pi/2.
            num_acceleration_levels (int, optional): [number of acceleration actions
                uniformly spaced between max and min acceleration]. Defaults to 10.
            num_turn_levels (int, optional): [number of turn actions uniformly spaced
                between max and min turns]. Defaults to 10.
            edge_hit_penalty (float, optional): [penalty for hitting the edge (wall)].
                Defaults to -0.0.
            use_full_observation (bool, optional): [boolean indicating whether to
                include all the agents' data in the observation or just the nearest
                neighbors]. Defaults to True.
            num_other_agents_observed (int, optional): [number of nearest neighbors
                in the obs (only takes effect when use_full_observation is False)].
                Defaults to 2.
            tagging_distance (float, optional): [margin between a
                tagger and runner to consider the runner as 'tagged'. This multiplies
                on top of the grid length]. Defaults to 0.01.
            tag_reward_for_tagger (float, optional): [positive reward for the tagger
                upon tagging a runner]. Defaults to 1.0
            step_penalty_for_tagger (float, optional): [penalty for every step
                the game goes on]. Defaults to -0.0.
            tag_penalty_for_runner (float, optional): [negative reward for getting
                tagged]. Defaults to -1.0
            step_reward_for_runner (float, optional): [reward for every step the
                runner isn't tagged]. Defaults to 0.0.
            end_of_game_reward_for_runner (float, optional): [reward at the end of
                the game for a runner that isn't tagged]. Defaults to 1.0.
            runner_exits_game_after_tagged (bool, optional): [boolean indicating
                whether runners exit the game after getting tagged or can remain in and
                continue to get tagged]. Defaults to True.
            env_backend (string, optional): [indicate whether to use the CPU
                or the GPU (either pycuda or numba) for stepping through the environment].
                Defaults to "cpu".
        """
        super().__init__()

        self.float_dtype = np.float32
        self.int_dtype = np.int32
        # small number to prevent indeterminate cases
        self.eps = self.float_dtype(1e-10)

        assert num_taggers > 0
        self.num_taggers = num_taggers

        assert num_runners > 0
        self.num_runners = num_runners

        self.num_agents = self.num_taggers + self.num_runners

        assert episode_length > 0
        self.episode_length = episode_length

        # Square 2D grid
        assert grid_length > 0
        self.grid_length = self.float_dtype(grid_length)
        self.grid_diagonal = self.grid_length * np.sqrt(2)

        # Penalty for hitting the edges
        assert edge_hit_penalty <= 0
        self.edge_hit_penalty = self.float_dtype(edge_hit_penalty)

        # Seeding
        self.np_random = np.random
        if seed is not None:
            self.seed(seed)

#iterate such that a group is added for num_policies. 

Fix number of agents depending on estimated total number of neurons. THen keep dividing until BIC stops improving. 

In [ ]:
"""


env_wrapper = EnvWrapper(
    TagContinuous(**run_config["env"]),
    num_envs=run_config["trainer"]["num_envs"],
    env_backend='pycuda',
)

# Agents can share policy models: this dictionary maps policy model names to agent ids.
policy_tag_to_agent_id_map = {
    "tagger": list(env_wrapper.env.taggers),
    "runner": list(env_wrapper.env.runners),
}

# Create the trainer object
trainer = Trainer(
    env_wrapper=env_wrapper,
    config=run_config,
    policy_tag_to_agent_id_map=policy_tag_to_agent_id_map,
)

# Perform training!
trainer.train()

# Shut off gracefully
trainer.graceful_close()


"""

In [3]:
env_wrapper = EnvWrapper(
    TagContinuous(**run_config["env"]),
    num_envs=run_config["trainer"]["num_envs"],
    env_backend='pycuda',
)

# Agents can share policy models: this dictionary maps policy model names to agent ids.
policy_tag_to_agent_id_map = {
    "tagger": list(env_wrapper.env.taggers),
    "runner": list(env_wrapper.env.runners),
}
"""
numPolicies= 3
for i in numPolicies:
    policy_tag_to_agent_id_map[str(i)] = list(env_wrapper.env.taggers),

"""
# Create the trainer object
trainer = Trainer(
    env_wrapper=env_wrapper,
    config=run_config,
    policy_tag_to_agent_id_map=policy_tag_to_agent_id_map,
)

# Perform training!
trainer.train()

# Shut off gracefully
trainer.graceful_close()

/root/miniconda3/envs/rapids-23.04/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_validation.py:114: UserWarning: WARNING: failed to get cudart_version from onnxruntime build info.
  warnings.warn("WARNING: failed to get cudart_version from onnxruntime build info.")




Device: 0
Iterations Completed                    : 1 / 50
Speed performance stats
Mean policy eval time per iter (ms)     :     249.74
Mean action sample time per iter (ms)   :      38.92
Mean env. step time per iter (ms)       :     147.29
Mean training time per iter (ms)        :     338.13
Mean total time per iter (ms)           :     786.96
Mean steps per sec (policy eval)        :   40041.14
Mean steps per sec (action sample)      :  256947.87
Mean steps per sec (env. step)          :   67891.39
Mean steps per sec (training time)      :   29574.42
Mean steps per sec (total)              :   12707.11
Metrics for policy 'runner'
VF loss coefficient                     :    0.01000
Entropy coefficient                     :    0.05000
Total loss                              :    0.62760
Policy loss                             :    0.86481
Value function loss                     :    0.24222
Mean rewards                            :    0.00151
Max. rewards                           

In [3]:
import torch
import numpy as np
import sys, copy, math, time, pdb
import os.path
import random
import pdb
import csv
import argparse
import itertools
from itertools import permutations, product
from sklearn.model_selection import train_test_split
import torch.optim as optim
from torchdiffeq import odeint
import itertools

import torch
from pettingzoo.mpe import simple_speaker_listener_v4
from tqdm import trange

from agilerl.components.multi_agent_replay_buffer import MultiAgentReplayBuffer
from agilerl.wrappers.pettingzoo_wrappers import PettingZooVectorizationParallelWrapper

from agilerl.algorithms.matd3 import MATD3

https://github.com/spxuw/DKI/blob/main/DKI.py

#import libraries 

In [2]:
import numpy as np
from gymnasium.spaces import Dict, Discrete
import argparse
import os

from ray import air, tune
from ray.air.constants import TRAINING_ITERATION
#from ray import rllib

each time the algorithm decides it grouping will be created 